#Problem.py


In [ ]:
import nltk
nltk.download('words')

In [ ]:
import random, math, numpy as np, copy, string
import time
import nltk
from random import randint, seed
from tqdm import tqdm
seed(0)
number_of_nodes_explored_min_conflict = 0
number_of_nodes_explored_backtrack = 0

nltk_words = nltk.corpus.words
english_words = nltk_words.words()

class Wordoku():
    def __init__(self, path):
        global number_of_nodes_explored_backtrack
        self.chars = []
        self.wordoku = []
        self.state = []
        number_of_nodes_explored_backtrack = -1
        self.meaningFullWords = set()
        self.__readString(path)

    #to set chars and storing initial state in wordoku
    def __readString(self, path):
        fp = open(path, 'r')
        Lines = fp.readlines()
        chars = set()
        for line in Lines:
            x = list(line)
            if(x[-1] == '\n'):
                x = x[:-1]
            self.wordoku.append(x)
            for ch in x:
                if ch != '*':
                    chars.add(ch)

        self.chars = list(chars)

    #to check whether current character is valid or not
    def isValid(self, row, col, ch):
      #checking in row
      for x in range(9):
          if self.wordoku[row][x] == ch:
              return False

      #checking in column
      for x in range(9):
          if self.wordoku[x][col] == ch:
              return False

      #checking in current matrix
      startRow = row - row % 3
      startCol = col - col % 3
      for i in range(3):
          for j in range(3):
              if self.wordoku[i + startRow][j + startCol] == ch:
                  return False
      return True

    #number of constraints
    def number_of_constraints(self, row_idx, col_idx, char):
        count = 0
        #calculating in row
        for j in range(9):
            if char in self.state[row_idx][j]:
                count += 1

        #checking in column
        for i in range(9):
            if char in self.state[i][col_idx]:
                count += 1

        #checking in current matrix
        startRow = row_idx - row_idx % 3
        startCol = col_idx - col_idx % 3
        for i in range(3):
            for j in range(3):
                if char in self.state[startRow + i][startCol + j]:
                    count+=1
        return count

    def find_next_empty_backtrack(self, state):
        min_idx = (0, 0)
        min_const = 100
        isFull = False
        for i in range(9):
            for j in range(9):
                if self.wordoku[i][j]=='*':
                    if len(state[i][j]) < min_const:
                        min_idx = (i, j)
                        min_const = len(state[i][j])
                        isFull = True
        if isFull == True:
            return min_idx[0], min_idx[1]
        else:
            return -1, -1

    def find_next_empty_min_conflict(self, epochs = 1000):
        constrained_vars = []
        for i in range(9):
            for j in range(9):
                if self.state[i][j] != '0':
                    temp = self.number_of_constraints(i, j, self.wordoku[i][j])
                    if temp != 0:
                        constrained_vars.append([i, j])
        if constrained_vars == []:
            return -1, -1
        else:
            pos = randint(0, len(constrained_vars) - 1)
            return constrained_vars[pos][0], constrained_vars[pos][1]

    def find_min_constrained_value(self, state, row_idx, col_idx):
        possible_chars = state[row_idx][col_idx]
        char_to_return = []
        constraint_values = []
        for char in possible_chars:
            constraints = self.number_of_constraints(row_idx, col_idx, char)
            constraint_values.append(constraints)
            char_to_return.append(char)
        idx = np.argsort(np.array(constraint_values))
        char_to_return2 = []
        for i in range(idx.shape[0]):
            char_to_return2.append(char_to_return[idx[i]])
        return char_to_return2

    #for constraints propogation
    def constraints_propogation(self, state, row_idx, col_idx, char):
        global number_of_nodes_explored_backtrack
        number_of_nodes_explored_backtrack +=1
        #in Row
        for j in range(9):
            state[row_idx][j] = state[row_idx][j].replace(char, '')
        #in Col
        for i in range(9):
            state[i][col_idx] = state[i][col_idx].replace(char, '')
        #in 3*3 matrix
        startRow = row_idx - row_idx % 3
        startCol = col_idx - col_idx % 3
        for i in range(3):
            for j in range(3):
                state[i + startRow][j + startCol] = state[i + startRow][j + startCol].replace(char, '')
        return state

    def min_conflict(self, row_idx, col_idx):
        curr_char = self.wordoku[row_idx][col_idx]

        curr_cnt = 1000

        _possible = copy.deepcopy(self.state[row_idx][col_idx]).replace(curr_char, '')

        for ch in _possible:
            self.wordoku[row_idx][col_idx] = ch
            cnt = self.number_of_constraints(row_idx, col_idx, ch)
            if(cnt < curr_cnt):
                curr_cnt = cnt
                curr_char = ch
        self.wordoku[row_idx][col_idx] = curr_char
        return curr_char

    def findMeaning(self, out_path):
        global english_words
        fp = open(out_path, 'w')
        for row in range(9):
            row_string = ''.join(self.wordoku[row])
            for i in range(9):
                for j in range(i + 1, 10):
                    substring1 = row_string[i : j].lower()
                    substring2 = row_string[j : i : -1].lower()
                    if(substring1 in english_words):
                        self.meaningFullWords.add(substring1)
                    if(substring2 in english_words):
                        self.meaningFullWords.add(substring2)

        col_list = []
        for col in range(9):
            col_string = ""
            for row in range(9):
                col_string += self.wordoku[row][col]
            col_list.append(col_string)
        for col in range(9):
            col_string = col_list[col]
            for i in range(9):
                for j in range(i + 1, 10):
                    substring1 = col_string[i : j].lower()
                    substring2 = col_string[j : i : -1].lower()
                    if(substring1 in english_words):
                        self.meaningFullWords.add(substring1)
                    if(substring2 in english_words):
                        self.meaningFullWords.add(substring2)
        diag1_string = ""
        diag2_string = ""
        for row in range(9):
                diag1_string += self.wordoku[row][row]
                diag2_string += self.wordoku[row][8 - row]

        for i in range(9):
            for j in range(i + 1, 10):
                substring1 = diag1_string[i : j].lower()
                substring2 = diag1_string[j : i : -1].lower()
                substring3 = diag2_string[i : j].lower()
                substring4 = diag2_string[j : i : -1].lower()
                if(substring1 in english_words):
                    self.meaningFullWords.add(substring1)
                if(substring2 in english_words):
                    self.meaningFullWords.add(substring2)
                if(substring3 in english_words):
                    self.meaningFullWords.add(substring3)
                if(substring4 in english_words):
                    self.meaningFullWords.add(substring4)

        meaning_full_words = sorted(self.meaningFullWords, key=lambda x: (len(x), x))

        for word in meaning_full_words:
            fp.write(word)
            fp.write('\n')


    def goal_test(self):
        for i in range(9):
            row_set = set()
            col_set = set()
            for j in range(9):
                row_set.add(self.wordoku[i][j])
                col_set.add(self.wordoku[j][i])
            if(len(list(row_set)) != 9):
                return False
            if(len(list(col_set)) != 9):
                return False
        for i in range(3):
            for j in range(3):
                box_set = set()
                for i1 in range(3):
                    for j1 in range(3):
                        box_set.add(self.wordoku[i * 3 + i1][j * 3 + j1])
                if((len(list(box_set))) != 9):
                    return False

        return True

    def printWordoku(self):
        for i in range(9):
            for j in range(9):
                print(self.wordoku[i][j], end = " ")
            print()

#BackTrack.py

In [ ]:
#Backtracking
class BackTrack(Wordoku):
    def __init__(self, path):
        super().__init__(path)
        self.__initState()

    def __initState(self):
        x = ''.join(self.chars)
        for i in range(9):
            temp = []
            for j in range(9):
                if self.wordoku[i][j] == '*':
                    temp.append(x)
                else:
                    temp.append('')
            self.state.append(temp)

        for i in range(9):
            for j in range(9):
                if self.wordoku[i][j] != '*':
                    self.state = self.constraints_propogation(self.state, i, j, self.wordoku[i][j])

    def solve(self, state):
        row, col = self.find_next_empty_backtrack(state)
        if row == -1 and col == -1:
            return True
        chars = self.find_min_constrained_value(state, row, col)

        for char in chars:

            self.wordoku[row][col] = char
            new_state = self.constraints_propogation(copy.deepcopy(state), row, col, char)

            if self.solve(copy.deepcopy(new_state)) == True:
                return True
            self.wordoku[row][col] = '*'

        return False

    def solveWordoku(self):
        self.solve(copy.deepcopy(self.state))

#Min_Conflict.py

In [ ]:
# Minimum Conflict Search
class MinConflict(Wordoku):
    def __init__(self, path):
        super().__init__(path)
        self.__initState()
        self.__initWordoku()

    def __initState(self):
        x = ''.join(self.chars)
        for i in range(9):
            temp = []
            for j in range(9):
                if self.wordoku[i][j] == '*':
                    temp.append(x)
                else:
                    temp.append('0')
            self.state.append(temp)

        for i in range(9):
            for j in range(9):
                if self.wordoku[i][j] == '0':
                    self.state = self.constraints_propogation(i, j, self.wordoku[i][j])

    def __initWordoku(self):
        for i in range(9):
            for j in range(9):
                if self.state[i][j] != '*':
                    idx = randint(0, len(self.state[i][j]) - 1)
                    self.wordoku[i][j] = self.state[i][j][idx]

    def solve(self):
        global number_of_nodes_explored_min_conflict
        number_of_nodes_explored_min_conflict += 1
        row, col = self.find_next_empty_min_conflict(self.state)
        if row == -1 and col == -1:
            return True
        x = self.min_conflict(row, col)

    def solveWordoku(self):
        for i in range(500):
            self.solve()
            if self.goal_test() == True:
                return True

#main.py


In [ ]:
#Initial state of problem
start1 = time.time()


path = '/content/input1.txt'
# path = '/content/input2.txt'
# path = '/content/input3.txt'
# path = '/content/input4.txt'
# path = '/content/input5.txt'


out_path = '/content/meaning_full_words1.txt'
# out_path = '/content/meaning_full_words2.txt'
# out_path = '/content/meaning_full_words3.txt'
# out_path = '/content/meaning_full_words4.txt'
# out_path = '/content/meaning_full_words5.txt'

In [ ]:
#BackTrack search with constraints propogation
game = BackTrack(path)
print("Initial State of Wordoku in Backtrack:")
game.printWordoku()
print()
start2 = time.time()
game.solveWordoku()
end2 = time.time() - start2
print("Goal state of Wordoku:")
game.printWordoku()
print()
game.findMeaning(out_path)
if(game.goal_test()):
    print("Correct Solution")
else:
    print("Incorrect Solution")
end1 = time.time() - start1
print("Total number of nodes explored are ", number_of_nodes_explored_backtrack)
print('Total Time: ', end1)
print('Search Time: ', end2)

In [ ]:
#Min-conflict search
game = MinConflict(path)
print("Initial State of Wordoku in Min-Conflict:")
game.printWordoku()
print()
start2 = time.time()
for i in tqdm(range(int(1000))):
    game = MinConflict(path)
    if game.solveWordoku() == True:
        break
end2 = time.time() - start2
print("Goal state of Wordoku:")
game.printWordoku()
print()
game.findMeaning(out_path)
if(game.goal_test()):
    print("Correct Solution")
else:
    print("Incorrect Solution")
end1 = time.time() - start1
print("Total number of nodes explored are ", number_of_nodes_explored_min_conflict)
print('Total Time: ', end1)
print('Search Time: ', end2)

#Analysis


For BackTracking with Constraints propogation:
*   Time Complexity : O(9^(k)) (where k is number of blank spaces)
*   Space Complexity : 0(n * n * n) (where n is dimension of wordoku)



For MinConflict

*   Time Complexity : algorithm cannot be defined it is based on probability.
*   Space Complexity : O(n * n * n) (where n is dimension of wordoku)



Comparision of Total time, Total search time and number of nodes generated
it is there in content in terms of pdf

Also all meaningfull words are generated in meaningFullWords{x}.txt file where x represent from which input file it generated and it will be stored in content folder in colab only.